In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import (
    _fetch_batch,
    fetch_data,
    fetch_spot_data,
)
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [2]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_finnifty(f_today):

    days_to_thursday = (1 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_midcpnifty(f_today):

    days_to_thursday = (0 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [3]:
bnf_pandas = pd.read_csv("../data/nifty.csv")

In [4]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [5]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [6]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))

In [7]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                pl.col("v").sum().alias("v"),
            ]
        )
    )

In [8]:
def generate_signals(df, signal_ma, candles_in_num):
    df["c"] = pd.to_numeric(df["c"], errors="coerce")
    df["Signal MA"] = df["c"].rolling(window=signal_ma).mean()
    # df["Trailing MA"] = df["c"].rolling(window=trailing_ma).mean()

    df["Sell Signal"] = 0

    # Generate signals using boolean masking
    sell_signal_mask = df["Signal MA"] <= df["Signal MA"].shift(1)

    for i in range(1, candles_in_num):
        sell_signal_mask &= df["Signal MA"].shift(i) <= df["Signal MA"].shift(i + 1)

    df.loc[sell_signal_mask, "Sell Signal"] = 1

    return df

In [9]:
data = bnf_pandas.copy()
# data['datetime'] = pd.to_datetime(data['datetime'].dt.date)
print(data["datetime"].head())
trading_days_set = set(data["datetime"].dt.date)
# sorted(trading_days_set)

0   2017-01-02 09:15:00
1   2017-01-02 09:16:00
2   2017-01-02 09:17:00
3   2017-01-02 09:18:00
4   2017-01-02 09:19:00
Name: datetime, dtype: datetime64[ns]


In [10]:
# GLOBAL VARIABLES

# INSTRUMENT = "BANKNIFTY"
# INDEX = "bnf"

# INSTRUMENT = "MIDCPNIFTY"
# INDEX = "midcpnifty"

# INSTRUMENT = "FINNIFTY"
# INDEX = "finnifty"

INSTRUMENT = "NIFTY"
INDEX = "nifty"

PORTFOLIO_VALUE = 10_00_000
INDEX_LEV = 7

In [11]:
data = data.reset_index(drop=True)
data.set_index("datetime", inplace=True)
data.tail()

open       high        low      close  volume
datetime                                                               
2024-06-07 15:25:00 23287.4000 23288.9000 23281.7000 23282.7500       0
2024-06-07 15:26:00 23282.2000 23282.8000 23274.4000 23279.4000       0
2024-06-07 15:27:00 23279.4000 23284.9000 23275.0500 23284.9000       0
2024-06-07 15:28:00 23282.8000 23283.5500 23271.0000 23271.5500       0
2024-06-07 15:29:00 23273.6500 23275.7000 23262.8000 23270.0500       0

In [29]:
async def trade_option_selling(df, index, strike, asset_class, expiry, signal_ma, no_of_candles, tf, sl):

    trade_book = []
    in_trade = False
    signal_entry_price = 100000
    signal_initial_sl = 0
    already_signal_exists = False
    is_trailing_active = False
    eod_exit_flag = False
    remark = ""
    entry_time = None
    no_more_trades = False

    df['datetime'] = pd.to_datetime(df['datetime'])
    # print(df.to_string())

    for i in range(0, len(df)):
        # print(df.iloc[i]['datetime'])
        if not no_more_trades:
            
            points = 0
            current_candle_open = df.iloc[i]["o"]
            current_candle_high = df.iloc[i]["h"]
            current_candle_low = df.iloc[i]["l"]
            current_candle_close = df.iloc[i]["c"]
            current_candle_volume = df.iloc[i]["v"]
            subset_df = df[max(0, i-signal_ma):i+1]

            if not in_trade:
                    
                if df.iloc[i]["Sell Signal"] == 1:
                    if not already_signal_exists:
                        # Fresh Sell Signal
    
                        # print(df[df['h'] == max(subset_df['h'])].to_string())
                        # print("Fresh Sell Signal")
                        signal_entry_price = current_candle_low
                        signal_initial_sl = current_candle_low * sl
                        # signal_initial_sl = max(subset_df['h'])
                        signal_creation_time = df.iloc[i]["datetime"]
                        already_signal_exists = True
                        # print(signal_creation_time)
    
                    else:
                        if current_candle_low > signal_entry_price:
                            # Better Candle
                            # print(df[df['h'] == max(subset_df['h'])].to_string())
                            # print("Better Candle")
                            signal_entry_price = current_candle_low
                            signal_initial_sl = current_candle_low * sl
                            # signal_initial_sl = max(subset_df['h'])
                            signal_creation_time = df.iloc[i]["datetime"]
                            # print(signal_entry_price, signal_initial_sl)
                            # print(signal_creation_time)
    
                        elif current_candle_low <= signal_entry_price:
    
                            if (
                                current_candle_open < signal_entry_price
                                and df.iloc[i]["datetime"].date()
                                > df.iloc[i - 1]["datetime"].date()
                            ):
                                # Gap Down Condition, Skip Entry
    
                                # print("Gap Down Open, Skip Trade")
                                already_signal_exists = False
                                signal_entry_price = 100000
                                signal_initial_sl = 0
    
                            else:
                                # Entry Triggered
        
                                # print("Entry Triggered")
                                in_trade = True
                                entry_time = df.iloc[i]["datetime"]
                                entry_price = signal_entry_price
                                # print(entry_price, signal_initial_sl)
                                points = 0
                                vol_entry = current_candle_volume
    
                else:
                    # Signal Does Not Exist In this Candle
                    if (df.iloc[i - 1]["Sell Signal"] == 1):
                        # print(df[df['h'] == max(subset_df['h'])].to_string())
                        # Condition to Enter the trade even if current candle is not Signal Candle
                        signal_entry_price = df.iloc[i - 1]["l"]
                        signal_initial_sl = df.iloc[i - 1]["l"] * sl
                        # signal_initial_sl = max(subset_df['h'])
                        signal_creation_time = df.iloc[i - 1]["datetime"]
                        already_signal_exists = True
                        # print(signal_entry_price, signal_initial_sl)
                        # print(signal_creation_time)
        
                        if current_candle_low <= signal_entry_price and df.iloc[i]['datetime'] != df.iloc[0]['datetime']:
                            # Entry Triggered
    
                            # print("Entry Triggered !")
                            in_trade = True
                            entry_time = df.iloc[i]["datetime"]
                            entry_price = signal_entry_price
                            points = 0
                            vol_entry = current_candle_volume
                            # print(entry_price, signal_initial_sl)
                        else:
                            # Discard Existing Signal
                            
                            # print('Signal Discarded')
                            already_signal_exists = False
                            signal_entry_price = 100000
                            signal_initial_sl = 0
        
            if in_trade:
                # print(df.iloc[i]['datetime'])
                # print('IN TRADE')
                trade_entry_price = signal_entry_price
                trade_initial_sl = signal_initial_sl
                trade_target = 1

                # print(trade_entry_price, trade_initial_sl)

                if trade_entry_price < trade_target:
                    in_trade = False
                    points = 0
                    # continue
                else:
                    if current_candle_open > trade_initial_sl:
                        if (
                            df.iloc[i]["datetime"].date() == entry_time.date()
                            and df.iloc[i]["datetime"].time() == entry_time.time()
                        ):
                            if current_candle_close >= trade_initial_sl:
                                # print('Initial SL Hit')
                                in_trade = False
                                points = -1 * (trade_initial_sl - trade_entry_price)
                                exit_price = trade_initial_sl
                                exit_time = df.iloc[i]["datetime"]
                                remark = "Initial SL hit"
                                max_price = df[:i+1]['h'].max()
                                min_price = df[:i+1]['l'].min()
                                # print(points)
        
                        else:
                            # Gap Open Outside ISL
        
                            # print(df.iloc[i])
                            # print('Gap Open Outside ISL')
                            in_trade = False
                            points = trade_entry_price - current_candle_close
                            exit_price = current_candle_close
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Gap Outside ISL"
                            max_price = df[:i+1]['h'].max()
                            min_price = df[:i+1]['l'].min()
                            # print(points)
        
                    elif current_candle_high >= trade_initial_sl:
                        # Initial SL Hit
        
                        # print('Initial SL Hit')
                        # print(current_candle_high, trade_initial_sl)
                        in_trade = False
                        points = -1 * (trade_initial_sl - trade_entry_price)
                        exit_price = trade_initial_sl
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Initial SL Hit"
                        max_price = df[:i+1]['h'].max()
                        min_price = df[:i+1]['l'].min()
                        # print(points)
        
                    elif current_candle_low <= trade_target:
                        # Target Hit
        
                        # print('Target Hit')
                        # print(current_candle_high, trade_initial_sl)
                        in_trade = False
                        points = -1 * (trade_target - trade_entry_price)
                        exit_price = trade_target
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Target Hit"
                        no_more_trades = True
                        max_price = df[:i+1]['h'].max()
                        min_price = df[:i+1]['l'].min()
                        # print('NO MORE TRADES')
                        # print(points)
        
                    elif (df.iloc[i]['datetime'].time() >= dt.time(15, 15)) and (df.iloc[i]['datetime'].date() == expiry):
                        # Expiry Exit
        
                        # print('Expiry Exit Condition Hit')
                        # print(current_candle_high, trade_initial_sl)
                        in_trade = False
                        points = -1 * (current_candle_close - trade_entry_price)
                        exit_price = current_candle_close
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Expiry Exit"
                        no_more_trades = True
                        max_price = df[:i+1]['h'].max()
                        min_price = df[:i+1]['l'].min()
                        # print('NO MORE TRADES')
                        # print(points)
                        
                    if points and (entry_price < 99999):
                        # print(entry_time)
                        if (entry_time.date() == expiry) and (entry_time.time() >= dt.time(14, 0)):
                            # print('IF CONDITION Triggered in if points:')
                            points = 0
                            in_trade = False
                            already_signal_exists = False
                            remark = ""
                            is_trailing_active = False
                            entry_time = None
                            # no_more_trades = False
                            # continue
                        
                        else:
                            # print('ELSE CONDITION Triggered in if points:')
                            # qty = int(round(portfolio_value * 5 / entry_price / 15)) * 15
                            qty = int(round(PORTFOLIO_VALUE * INDEX_LEV / strike / 15)) * 15
                            # rpt_qty = PORTFOLIO_VALUE * 0.01 / abs(entry_price - trade_initial_sl)
                            # qty = min(max_qty, rpt_qty)
                            slippage = 0.01 * (entry_price + exit_price)
                            # slippage = 10
                            final_points = points - slippage
                            # final_points = points

                            # if vol_entry * strike < 250000000:
                            #     vol_remark = 'ILLIQUID'
                            # elif vol_entry * strike >= 250000000:
                            #     vol_remark = 'LIQUID'z
                            
                            trade = {
                                "Index": index,
                                "Strike": strike,
                                "Option Type": asset_class,
                                "Expiry": expiry,
                                "Signal Generated At": signal_creation_time,
                                "Trade Type": "SELL",
                                "Entry Date": entry_time.date(),
                                "Entry Time": entry_time.time(),
                                "Entry Price": entry_price,
                                "Initial SL": trade_initial_sl,
                                "Target": trade_target,
                                "Exit Time": exit_time,
                                "Exit Price": exit_price,
                                "Points Captured": points,
                                "Slippages": slippage,
                                "After Costs": final_points,
                                "PnL": final_points * qty,
                                "Remarks": remark,
                                # 'Volume at Entry Candle': vol_entry,
                                # 'Volume Remark': vol_remark,
                                # "Max Qty": max_qty,
                                # "RPT Qty": rpt_qty,
                                'Qty': qty,
                                # "Leverage": "5x",
                                "ROI%": (final_points * qty / PORTFOLIO_VALUE) * 100,
                                "Max Price": max_price,
                                "Min Price": min_price,
                                "Trade Year": entry_time.year,
                                "Trade Month": entry_time.month,
                                "Variation": f"{signal_ma}, {no_of_candles}, {tf}, {sl}% SL",
                            }
                            # print(trade)
                            trade_book.append(trade)
                            # print("Appended : \n", trade)
                            points = 0
                            in_trade = False
                            already_signal_exists = False
                            remark = ""
                            is_trailing_active = False
                            entry_time = None
                            # no_more_trades = False
                    
        # if no_more_trades:
    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [13]:
# async def convert_to_weekly(df):

#     # FOR BNF 

#     before_df = df[df.index.date < dt.date(2023, 8, 31)]
#     after_df = df[df.index.date >= dt.date(2023, 8, 31)]

#     before_df_resampled = before_df.resample(
#         "W-THU", label="right", closed="right"
#     ).agg(
#         {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"}
#     )

#     after_df_resampled = after_df.resample(
#         "W-WED", label="right", closed="right"
#     ).agg(
#         {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"}
#     )

#     weekly_data = pd.concat([before_df_resampled, after_df_resampled])

#     # Reset index if you want to make the DateTime a column again
#     weekly_data = weekly_data.reset_index()

#     return weekly_data



async def convert_to_weekly(df):

    # FOR FNF, NIFTY, MIDCP

    df_resampled = df.resample(
        "W-THU", label="right", closed="right"
    ).agg(
        {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"}
    )

    weekly_data = pd.DataFrame(df_resampled)
    weekly_data = weekly_data.reset_index()

    return weekly_data

weekly_data = await convert_to_weekly(data)

In [14]:
weekly_data["datetime"] = pd.to_datetime(weekly_data["datetime"])
weekly_data["Week Start"] = weekly_data["datetime"].dt.date - dt.timedelta(days=6)
weekly_data["Expiry"] = weekly_data["datetime"].dt.date

for i in range(0, len(weekly_data)):
    current_expiry = weekly_data["Expiry"].iloc[i]
    if nse.valid_days(start_date=current_expiry, end_date=current_expiry).empty:
        weekly_data["Expiry"].iloc[i] = current_expiry - dt.timedelta(days=1)
print(weekly_data.to_string())
# await trade(weekly_data)

      datetime       open       high        low      close  volume  Week Start      Expiry
0   2017-01-05  8210.1000  8282.5500  8134.3000  8270.5000       0  2016-12-30  2017-01-05
1   2017-01-12  8283.0000  8417.0000  8228.0000  8409.1500       0  2017-01-06  2017-01-12
2   2017-01-19  8457.6500  8460.2000  8373.1500  8431.2000       0  2017-01-13  2017-01-19
3   2017-01-26  8404.3500  8612.1500  8328.0500  8597.5000       0  2017-01-20  2017-01-25
4   2017-02-02  8610.5000  8756.9000  8537.8000  8722.3000       0  2017-01-27  2017-02-02
5   2017-02-09  8735.1500  8821.2000  8707.7500  8786.5500       0  2017-02-03  2017-02-09
6   2017-02-16  8812.3500  8826.4500  8713.0500  8777.8000       0  2017-02-10  2017-02-16
7   2017-02-23  8883.7000  8981.9000  8804.7500  8941.2000       0  2017-02-17  2017-02-23
8   2017-03-02  8943.7000  8992.3500  8868.1000  8893.9000       0  2017-02-24  2017-03-02
9   2017-03-09  8885.5500  8977.7500  8860.3000  8927.8000       0  2017-03-03  2017-03-09

/tmp/ipykernel_89060/2745470051.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  weekly_data["Expiry"].iloc[i] = current_expiry - dt.timedelta(days=1)
/tmp/ipykernel_89060/2745470051.py:8: SettingWithCopyWarning: 
A value is trying to be 

In [30]:
async def trade(df, signal_ma, no_of_candles, tf, sl):

    combined_trades_ce = pd.DataFrame()
    combined_trades_pe = pd.DataFrame()
    total_trades = pd.DataFrame()

    if tf == "10m":
        ofs = "5m"
    elif tf == "30m":
        ofs = "15m"
    else:
        ofs = "0m"

    # print(df.tail().to_string())
    # for i in range(300, len(df)):
    for i in range(0, len(df)):
        current_date = df["Week Start"].iloc[i]
        current_expiry = df["Expiry"].iloc[i]
        # print(i, current_date)
        spot_open = df["open"].iloc[i]
        spot_atm = int(round(spot_open / 50) * 50)
        # print(spot_open, spot_atm)

        # current_expiry = await get_expiry(current_expiry - dt.timedelta(days=1))
        if current_expiry == dt.date(2024, 4, 17):
            current_expiry = dt.date(2024, 4, 16)
        if current_expiry == dt.date(2024, 5, 1):
            current_expiry = dt.date(2024, 4, 30)

        # print(current_expiry)

        ce_df = await fetch_data(
            index=INDEX,
            start_date=current_date,
            end_date=current_expiry,
            start_time=dt.time(9, 15),
            end_time=dt.time(15, 30),
            expiry=current_expiry,
            strike=spot_atm,
            asset_class="C",
        )
        # print(ce_df)
        if not isinstance(ce_df, str) and ce_df is not None:
            ce_df = ce_df.select(["datetime", "o", "h", "l", "c", "v"])
            ce_df = resample(ce_df, tf, offset=ofs)
            ce_df_pandas = ce_df.to_pandas()
            ce_df_pandas = ce_df_pandas[~(ce_df_pandas['datetime'].dt.time == pd.to_datetime('15:30').time())]
            ce_df = generate_signals(ce_df_pandas, signal_ma, no_of_candles)
            # print(spot_atm, 'CE DF :\n')
            # print(ce_df.to_string())

        else:
            continue

        pe_df = await fetch_data(
            index=INDEX,
            start_date=current_date,
            end_date=current_expiry,
            start_time=dt.time(9, 15),
            end_time=dt.time(15, 30),
            expiry=current_expiry,
            strike=spot_atm,
            asset_class="P",
        )
        # print(pe_df)
        if not isinstance(pe_df, str) and pe_df is not None:
            pe_df = pe_df.select(["datetime", "o", "h", "l", "c", "v"])
            pe_df = resample(pe_df, tf, offset=ofs)
            pe_df_pandas = pe_df.to_pandas()
            pe_df_pandas = pe_df_pandas[~(pe_df_pandas['datetime'].dt.time == pd.to_datetime('15:30').time())]
            pe_df = generate_signals(pe_df_pandas, signal_ma, no_of_candles)
            # print(spot_atm, 'PE DF :\n')
            # print(pe_df.to_string())

        # print('Executing CE Trades')
        ce_trades = await trade_option_selling(
            df=ce_df,
            index=INDEX,
            strike=spot_atm,
            asset_class="C",
            expiry=current_expiry,
            signal_ma=signal_ma,
            no_of_candles=no_of_candles,
            tf=tf,
            sl=sl,
        )

        combined_trades_ce = pd.concat([combined_trades_ce, ce_trades], ignore_index=True)
        # print(combined_trades_ce.to_string())

        # print('Executing PE Trades')
        pe_trades = await trade_option_selling(
            df=pe_df,
            index=INDEX,
            strike=spot_atm,
            asset_class="P",
            expiry=current_expiry,
            signal_ma=signal_ma,
            no_of_candles=no_of_candles,
            tf=tf,
            sl=sl,
        )

        combined_trades_pe = pd.concat([combined_trades_pe, pe_trades], ignore_index=True)
        # print(combined_trades_pe.to_string())

    total_trades = pd.concat([total_trades, combined_trades_ce, combined_trades_pe], ignore_index=True)
    total_trades = total_trades.sort_values(by="Signal Generated At", ignore_index=True)
    return total_trades 

In [31]:
# weekly_data

In [32]:
def generate_stats(tb_expiry, signal_ma, no_of_candles, tf, sl):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f'{signal_ma} , {no_of_candles}, {tf}, {sl}'
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [33]:
# # SIMULATION

# x = [15, 20, 25, 30, 35, 40, 45, 50]
# y = [1, 2, 3, 4, 5]
# z = ["3m", "5m", "10m", "15m", "30m"]
# sl = [1.25, 1.5, 1.75, 2]

# stats_dictionary = {}
# for i in x:
#     for j in y:
#         for k in z:
#             for l in sl:
#                 print(f"Signal MA : {i} , No. of Candles : {j} , TimeFrame : {k} , SL {l}")
#                 tb = await trade(weekly_data, i, j, k, l)
#                 # print(len(tb))
#                 if len(tb) > 0:
#                     tb_ce = tb[tb['Option Type'] == 'C']
#                     tb_pe = tb[tb['Option Type'] == 'P']
#                     stats = generate_stats(tb, i, j, k, l)
#                     stats_ce = generate_stats(tb_ce, i, j, k, l)
#                     stats_pe = generate_stats(tb_pe, i, j, k, l)
    
#                     for overall_roi_dd_ratio, stats_df in stats.items():
#                         if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 10:
#                             print("Overall Combined")
#                             print(stats_df.to_string())
#                             stats_dictionary[overall_roi_dd_ratio] = stats_df
    
#                     # for overall_roi_dd_ratio, stats_df in stats_ce.items():
#                     #     if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 5:
#                     #         print("Only CE")
#                     #         print(stats_df.to_string())
#                     #         stats_dictionary[overall_roi_dd_ratio] = stats_df
    
#                     # for overall_roi_dd_ratio, stats_df in stats_pe.items():
#                     #     if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 5:
#                     #         print("Only PE")
#                     #         print(stats_df.to_string())
#                     #         stats_dictionary[overall_roi_dd_ratio] = stats_df

In [34]:
# sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
# # tb = await trade(weekly_data, i, j, k)
# # tb

In [35]:
# sorted_stats

In [36]:
tb = await trade(weekly_data, 15, 5, "30m", 1.5)
# tb

In [37]:
tb

Index  Strike Option Type      Expiry Signal Generated At Trade Type  \
0    nifty    8400           C  2017-01-25 2017-01-23 12:15:00       SELL   
1    nifty    8400           P  2017-01-25 2017-01-24 09:45:00       SELL   
2    nifty    8900           P  2017-02-23 2017-02-20 12:15:00       SELL   
3    nifty    9100           C  2017-03-30 2017-03-27 12:45:00       SELL   
4    nifty    9100           C  2017-03-30 2017-03-27 14:15:00       SELL   
5    nifty    9100           P  2017-03-30 2017-03-28 11:45:00       SELL   
6    nifty    9200           P  2017-04-27 2017-04-24 12:15:00       SELL   
7    nifty    9450           C  2017-05-25 2017-05-22 12:15:00       SELL   
8    nifty    9450           P  2017-05-25 2017-05-22 15:15:00       SELL   
9    nifty    9450           P  2017-05-25 2017-05-25 11:45:00       SELL   
10   nifty    9650           C  2017-06-29 2017-06-27 12:15:00       SELL   
11   nifty    9650           P  2017-06-29 2017-06-29 10:15:00       SELL   
12   nifty    9900           P  2017-07-27 2017-07-24 12:45:00       SELL   
13   nifty    9900           C  2017-08-31 2017-08-29 12:15:00       SELL   
14   nifty    9900           C  2017-08-31 2017-08-30 09:45:00       SELL   
15   nifty    9900           P  2017-08-31 2017-08-30 13:45:00       SELL   
16   nifty    9900           C  2017-08-31 2017-08-31 12:15:00       SELL   
17   nifty   10100           C  2017-09-28 2017-09-25 12:15:00       SELL   
18   nifty   10200           P  2017-10-26 2017-10-24 13:45:00       SELL   
19   nifty   10350           C  2017-11-30 2017-11-27 12:15:00       SELL   
20   nifty   10350           P  2017-11-30 2017-11-28 09:45:00       SELL   
21   nifty   10350           P  2017-11-30 2017-11-28 14:15:00       SELL   
22   nifty   10350           C  2017-11-30 2017-11-29 10:15:00       SELL   
23   nifty   10450           P  2017-12-28 2017-12-26 12:15:00       SELL   
24   nifty   10450           C  2017-12-28 2017-12-28 10:45:00       SELL   
25   nifty   10850           P  2018-01-25 2018-01-22 12:15:00       SELL   
26   nifty   10850           C  2018-01-25 2018-01-25 12:45:00       SELL   
27   nifty   10600           C  2018-02-22 2018-02-19 12:15:00       SELL   
28   nifty   10600           P  2018-02-22 2018-02-20 13:45:00       SELL   
29   nifty    9950           P  2018-03-28 2018-03-26 12:15:00       SELL   
30   nifty    9950           C  2018-03-28 2018-03-28 12:15:00       SELL   
31   nifty   10550           P  2018-04-26 2018-04-23 12:45:00       SELL   
32   nifty   10550           C  2018-04-26 2018-04-25 10:45:00       SELL   
33   nifty   10550           P  2018-04-26 2018-04-26 12:15:00       SELL   
34   nifty   10550           P  2018-05-31 2018-05-28 13:15:00       SELL   
35   nifty   10550           C  2018-05-31 2018-05-29 13:45:00       SELL   
36   nifty   10550           P  2018-05-31 2018-05-31 09:45:00       SELL   
37   nifty   10750           P  2018-06-28 2018-06-25 12:45:00       SELL   
38   nifty   10750           C  2018-06-28 2018-06-26 09:45:00       SELL   
39   nifty   10750           C  2018-06-28 2018-06-26 15:15:00       SELL   
40   nifty   10750           P  2018-06-28 2018-06-27 09:15:00       SELL   
41   nifty   10950           P  2018-07-26 2018-07-23 12:45:00       SELL   
42   nifty   11550           P  2018-08-30 2018-08-27 12:15:00       SELL   
43   nifty   11550           C  2018-08-30 2018-08-29 13:45:00       SELL   
44   nifty   11250           C  2018-09-27 2018-09-24 12:45:00       SELL   
45   nifty   11250           C  2018-09-27 2018-09-25 10:45:00       SELL   
46   nifty   11250           P  2018-09-27 2018-09-26 11:45:00       SELL   
47   nifty   11250           C  2018-09-27 2018-09-26 12:45:00       SELL   
48   nifty   11250           C  2018-09-27 2018-09-26 14:45:00       SELL   
49   nifty   10350           C  2018-10-25 2018-10-22 12:15:00       SELL   
50   nifty   10550           P  2018-11-29 2018-11-27 12:15:00       SELL   
5

In [26]:
# tb_ce2 = tb[tb['Option Type'] == 'C']
# tb_pe2 = tb[tb['Option Type'] == 'P']

In [38]:
stats2 = generate_stats(tb, 15, 5, "30m", 1.5)
roi_overall, stats_overall = next(iter(stats2.items()))
stats_overall

Total ROI Total Trades Win Rate Avg Profit% per Trade  \
2017       5.2577           25  44.0000                2.0827   
2018       3.7186           28  53.5714                1.4639   
2019      32.2225          108  50.0000                2.3539   
2020     187.5867          115  56.5217                5.2424   
2021      94.8191          131  48.0916                3.2053   
2022      55.3794          121  52.0661                2.9798   
2023      -7.6389          118  47.4576                1.4263   
2024       4.8431           57  43.8596                2.3402   
Overall  376.1882          703  50.0711                2.9567   

        Avg Loss% per Trade Max Drawdown ROI/DD Ratio         Variation  
2017                -1.2608      -6.1106       0.8604  15 , 5, 30m, 1.5  
2018                -1.4031      -7.6686       0.4849  15 , 5, 30m, 1.5  
2019                -1.7572     -12.4908       2.5797  15 , 5, 30m, 1.5  
2020                -3.0633     -20.0631       9.3498  15 , 5, 30m, 1.5  
2021                -1.5752     -15.9272       5.9533  15 , 5, 30m, 1.5  
2022                -2.2818     -19.9646       2.7739  15 , 5, 30m, 1.5  
2023                -1.4115     -16.0286      -0.4766  15 , 5, 30m, 1.5  
2024                -1.6770     -10.6159       0.4562  15 , 5, 30m, 1.5  
Overall             -1.8934     -20.0631      18.7503  15 , 5, 30m, 1.5

In [ ]:
# tb.to_csv('PSM nifty_18_8.csv')

In [312]:
# tb_ill = (tb[(tb['Volume Remark'] == 'ILLIQUID') & (tb['Trade Year'] >= 2020)])
# # tb_ill['ROI%'].sum()
# tb_ill

In [313]:
# tb[(tb['Entry Price'] > 700) & (tb['Volume Remark'] == 'LIQUID')]

In [314]:
# len(tb[tb['Qty Traded'] == tb['Max Qty']])

In [34]:
tb2 = await trade(weekly_data, 35, 4, "15m", 1.25)
# tb2

21750 CE DF :

               datetime        o        h        l        c         v  Signal MA  Sell Signal
0   2023-12-29 09:15:00 249.7500 249.7500 154.3000 177.3000   4464153        NaN            0
1   2023-12-29 09:30:00 176.4000 195.5000 168.5000 169.7500   4787550        NaN            0
2   2023-12-29 09:45:00 169.5500 173.1000 153.1500 154.6000   2875200        NaN            0
3   2023-12-29 10:00:00 154.8500 166.3500 124.0000 125.2000   4771800        NaN            0
4   2023-12-29 10:15:00 126.0500 156.2000 122.7500 153.5500   4208650        NaN            0
5   2023-12-29 10:30:00 152.8500 168.7500 151.8500 162.2000   3235850        NaN            0
6   2023-12-29 10:45:00 161.8000 172.1500 148.9000 155.7000   2929300        NaN            0
7   2023-12-29 11:00:00 156.1000 157.7000 146.0500 154.1500   1813400        NaN            0
8   2023-12-29 11:15:00 153.7000 169.9500 153.0000 167.5500   2770800        NaN            0
9   2023-12-29 11:30:00 168.0000 177.0500 161

In [35]:
tb2

Index  Strike Option Type      Expiry Signal Generated At Trade Type  \
0   nifty   21750           P  2024-01-04 2024-01-01 12:30:00       SELL   
1   nifty   21750           C  2024-01-04 2024-01-01 13:15:00       SELL   
2   nifty   21750           C  2024-01-04 2024-01-02 09:30:00       SELL   
3   nifty   21750           P  2024-01-04 2024-01-04 11:45:00       SELL   
4   nifty   21700           C  2024-01-11 2024-01-08 12:30:00       SELL   
5   nifty   21700           C  2024-01-11 2024-01-09 09:30:00       SELL   
6   nifty   21700           P  2024-01-11 2024-01-09 11:15:00       SELL   
7   nifty   21700           C  2024-01-11 2024-01-09 11:45:00       SELL   
8   nifty   21700           P  2024-01-11 2024-01-11 09:45:00       SELL   
9   nifty   21700           P  2024-01-11 2024-01-11 11:00:00       SELL   
10  nifty   21700           P  2024-01-11 2024-01-11 11:45:00       SELL   
11  nifty   21750           P  2024-01-18 2024-01-15 12:30:00       SELL   
12  nifty   21750           C  2024-01-18 2024-01-16 14:15:00       SELL   
13  nifty   21600           P  2024-01-25 2024-01-20 12:30:00       SELL   
14  nifty   21600           C  2024-01-25 2024-01-20 13:15:00       SELL   
15  nifty   21600           P  2024-01-25 2024-01-20 15:00:00       SELL   
16  nifty   21600           C  2024-01-25 2024-01-23 11:00:00       SELL   
17  nifty   21600           P  2024-01-25 2024-01-24 15:00:00       SELL   
18  nifty   21600           P  2024-01-25 2024-01-25 11:00:00       SELL   
19  nifty   21450           P  2024-02-01 2024-01-30 12:45:00       SELL   
20  nifty   21450           C  2024-02-01 2024-01-30 13:45:00       SELL   
21  nifty   21450           P  2024-02-01 2024-01-31 10:45:00       SELL   
22  nifty   21800           C  2024-02-08 2024-02-05 13:00:00       SELL   
23  nifty   21800           P  2024-02-08 2024-02-06 11:45:00       SELL   
24  nifty   21800           C  2024-02-08 2024-02-07 12:30:00       SELL   
25  nifty   21800           C  2024-02-08 2024-02-08 10:15:00       SELL   
26  nifty   21750           C  2024-02-15 2024-02-12 12:30:00       SELL   
27  nifty   21750           C  2024-02-15 2024-02-13 11:00:00       SELL   
28  nifty   21750           P  2024-02-15 2024-02-13 13:30:00       SELL   
29  nifty   21750           P  2024-02-15 2024-02-14 11:30:00       SELL   
30  nifty   22000           P  2024-02-22 2024-02-19 12:45:00       SELL   
31  nifty   22000           P  2024-02-22 2024-02-20 09:15:00       SELL   
32  nifty   22000           P  2024-02-22 2024-02-20 10:00:00       SELL   
33  nifty   22000           C  2024-02-22 2024-02-20 14:30:00       SELL   
34  nifty   22000           C  2024-02-22 2024-02-20 15:15:00       SELL   
35  nifty   22300           C  2024-02-29 2024-02-26 12:30:00       SELL   
36  nifty   22300           C  2024-02-29 2024-02-26 13:15:00       SELL   
37  nifty   22300           P  2024-02-29 2024-02-27 13:00:00       SELL   
38  nifty   22300           P  2024-02-29 2024-02-27 14:00:00       SELL   
39  nifty   22050           P  2024-03-07 2024-03-04 11:00:00       SELL   
40  nifty   22050           C  2024-03-07 2024-03-05 10:00:00       SELL   
41  nifty   22050           P  2024-03-07 2024-03-05 10:30:00       SELL   
42  nifty   22050           P  2024-03-07 2024-03-05 11:30:00       SELL   
43  nifty   22050           C  2024-03-07 2024-03-05 13:00:00       SELL   
44  nifty   22500           C  2024-03-14 2024-03-12 12:30:00       SELL   
45  nifty   22050           P  2024-03-21 2024-03-18 12:45:00       SELL   
46  nifty   22050           C  2024-03-21 2024-03-19 09:45:00       SELL   
47  nifty   22050           P  2024-03-21 2024-03-20 13:00:00       SELL   
48  nifty   22050           P  2024-03-21 2024-03-20 14:00:00       SELL   
49  nifty   21950           P  2024-03-28 2024-03-26 12:30:00       SELL   
50  nifty   21950           C  2024-03-28 2024-03-26 13:15:00       SELL   
51  nifty   21950           P  2024-03-28 2024-03-26

In [59]:
midcp = await trade(weekly_data, 5, 5, "10m", 0)
stats2 = generate_stats(midcp, 5, 5, "10m", 0)
roi_overall, stats_overall = next(iter(stats2.items()))
stats_overall

Total ROI Total Trades Win Rate Avg Profit% per Trade  \
2017       0.0000            0      NaN                   NaN   
2018       0.0000            0      NaN                   NaN   
2019       0.0000            0      NaN                   NaN   
2020       0.0000            0      NaN                   NaN   
2021       0.0000            0      NaN                   NaN   
2022       0.0000            0      NaN                   NaN   
2023      84.7532           52  40.3846                7.7734   
2024     245.9328           77  41.5584               12.9610   
Overall  330.6860          129  41.0853               10.9056   

        Avg Loss% per Trade Max Drawdown ROI/DD Ratio      Variation  
2017                    NaN          NaN          NaN  5 , 5, 10m, 0  
2018                    NaN          NaN          NaN  5 , 5, 10m, 0  
2019                    NaN          NaN          NaN  5 , 5, 10m, 0  
2020                    NaN          NaN          NaN  5 , 5, 10m, 0  
2021                    NaN          NaN          NaN  5 , 5, 10m, 0  
2022                    NaN          NaN          NaN  5 , 5, 10m, 0  
2023                -2.5319     -19.5426       4.3368  5 , 5, 10m, 0  
2024                -3.7516     -33.2443       7.3977  5 , 5, 10m, 0  
Overall             -3.2541     -33.2443       9.9472  5 , 5, 10m, 0

In [60]:
midcp.head()

Index  Strike Option Type      Expiry Signal Generated At Trade Type  \
0  midcpnifty    8775           P  2023-09-04 2023-09-01 11:25:00       SELL   
1  midcpnifty    8975           P  2023-09-11 2023-09-05 10:55:00       SELL   
2  midcpnifty    8975           C  2023-09-11 2023-09-05 13:15:00       SELL   
3  midcpnifty    8975           C  2023-09-11 2023-09-06 13:15:00       SELL   
4  midcpnifty    8975           C  2023-09-11 2023-09-07 12:05:00       SELL   

   Entry Date Entry Time  Entry Price  Initial SL  Target           Exit Time  \
0  2023-09-01   11:35:00       2.9500      6.2000       1 2023-09-01 15:05:00   
1  2023-09-05   11:05:00      40.4000     48.0000       1 2023-09-08 09:25:00   
2  2023-09-05   13:25:00      75.6500     82.0500       1 2023-09-05 13:45:00   
3  2023-09-06   13:25:00      96.8000    105.6500       1 2023-09-06 14:15:00   
4  2023-09-07   12:15:00     102.0500    110.7500       1 2023-09-07 14:25:00   

   Exit Price  Points Captured  Slippages  After Costs         PnL  \
0      1.0000           1.9500     0.0395       1.9105   2607.8325   
1      1.0000          39.4000     0.4140      38.9860  52046.3100   
2     82.0500          -6.4000     1.5770      -7.9770 -10649.2950   
3    105.6500          -8.8500     2.0245     -10.8745 -14517.4575   
4    116.7500         -14.7000     2.1880     -16.8880 -22545.4800   

           Remarks   Qty    ROI%  Max Price  Min Price  Trade Year  \
0       Target Hit  1365  0.2608    11.2500     0.9500        2023   
1       Target Hit  1335  5.2046    64.9500     1.0000        2023   
2   Initial SL Hit  1335 -1.0649    89.1000    60.3500        2023   
3   Initial SL Hit  1335 -1.4517   122.3000    60.3500        2023   
4  Gap Outside ISL  1335 -2.2545   123.1000    60.3500        2023   

   Trade Month         Variation  
0            9  5, 5, 10m, 0% SL  
1            9  5, 5, 10m, 0% SL  
2            9  5, 5, 10m, 0% SL  
3            9  5, 5, 10m, 0% SL  
4            9  5, 5, 10m, 0% SL

In [61]:
midcp.to_csv('PSM midcp.csv')